# Convolutional Neural Network to classify between Cats & Dogs using Keras

You can get the dataset from [here](https://www.kaggle.com/c/dogs-vs-cats/data). Following should be the directory structure.

```
project   
│
└───dataset
│   │
│   └───single_prediction
│   │   │   cat_or_dog1.jpg
│   │   │   cat_or_dog2.jpg
│   │
│   │
│   └───test_set
│   │   │
│   │   └───cats
│   │   │   cat.1.jpg
│   │   │   cat.2.jpg
│   │   │   ...
│   │   │
│   │   └───dogs
│   │       dog.1.jpg
│   │       dog.2.jpg
│   │       ...
│   │  
│   └───training_set
│       │
│       └───cats
│       │   cat.1.jpg
│       │   cat.2.jpg
│       │   ...
│       │
│       └───dogs
│           dog.1.jpg
│           dog.2.jpg
│           ...
│     
└───script.py

```

We will start by importing the modules we will use

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Now we will define the architecture of our Convolutional Neural Network. I am currenty training on CPU so I'll define 2 convolution layers.

In [ ]:
# Adding convolutional layer
classifier.add(Conv2D(64, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Fattening images
classifier.add(Flatten())

# Adding fully connected layers
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Compiling our CNN.

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Now let's load our dataset

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Convolutional_Neural_Networks/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('Convolutional_Neural_Networks/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Now I will create a checkpoint to save the weights so that if you decide it to train the cnn further or make predicitons, we will be able to do so.

In [ ]:
checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)

classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000,
                         callbacks=[checkpointer])

Now let's save our trained model

In [ ]:
classifier.save('animal_classifier.h5')

Okay great, let's load the weights from our trained cnn.

In [ ]:
classifier.load_weights('/tmp/weights.hdf5')

Now let's make some predictions. I've saved some new images in single_prediction folders.

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('Convolutional_Neural_Networks/dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)